![](files/images/EscUpmPolit_p.gif "UPM")

<header style="width:100%;position:relative">
  <div style="width:80%;float:right;">
    <h1>Course Notes for Learning Intelligent Systems</h1>
    <h3>Department of Telematic Engineering Systems</h3>
    <h5>Universidad Politécnica de Madrid. © Carlos A. Iglesias </h5>
  </div>
        <img style="width:15%;" src="../logo.jpg" alt="UPM" />
</header>

## Introduction

This lecture provides an introduction to RDF and the SPARQL query language.

This is the first in a series of notebooks about SPARQL, which consists of:

* This notebook, which explains basic concepts of RDF and SPARQL
* [A notebook](01_SPARQL_Introduction.ipynb) that provides an introduction to SPARQL through a collection of  exercises of increasing difficulty.
* [An optional notebook](02_SPARQL_Custom_Endpoint.ipynb) with queries to a custom dataset.
The dataset is meant to be done after the [RDF exercises](../rdf/RDF.ipynb) and it is out of the scope of this course.
You can consult it if you are interested.

# RDF basics
This section is taken from [[1](#1), [2](#2)].

RDF allows us to make statements about resources. The format of these statements is simple. A statement always has the following structure:

```
<subject> <predicate> <object>
```
    
An RDF statement expresses a relationship between two resources. The **subject** and the **object** represent the two resources being related; the **predicate** represents the nature of their relationship.
The relationship is phrased in a directional way (from subject to object).
In RDF this relationship is known as a **property**.
Because RDF statements consist of three elements they are called **triples**.

Here are some examples of RDF triples (informally expressed in pseudocode):

```turtle
<Bob> <is a> <person>.
<Bob> <is a friend of> <Alice>.
```
      
Resources are identified by [IRIs](https://en.wikipedia.org/wiki/Internationalized_Resource_Identifier), which can appear in all three positions of a triple. For example, the IRI for Leonardo da Vinci in DBpedia is:

      <http://dbpedia.org/resource/Leonardo_da_Vinci>

IRIs can be abbreviated as *prefixed names*. For example, 
```turtle
PREFIX dbr: <http://dbpedia.org/resource/>

<dbr:Leonardo_da_Vinci>
```
     
Objects can be literals: 
 * strings (e.g., "plain string" or "string with language"@en)
 * numbers (e.g., "13.4"^^xsd:float)
 * dates (e.g., )
 * booleans
 * etc.
 
RDF data is stored in RDF repositories that expose SPARQL endpoints.
Let's query one of the most famous RDF repositories: [dbpedia](https://wiki.dbpedia.org/).
First, we should learn how to execute SPARQL in a notebook.

# Executing SPARQL in a notebook
There are several ways to execute SPARQL in a notebook.
Some of the most popular are:

* using libraries such as [sparql-client](https://pypi.org/project/sparql-client/) or [rdflib](https://rdflib.dev/sparqlwrapper/) that enable executing SPARQL within a Python3 kernel
* using other libraries. In our case, a light library has been developed (the file helpers.py) for accessing SPARQL endpoints using an HTTP connection.
* using the [graph notebook package](https://pypi.org/project/graph-notebook/)
* using a SPARQL kernel [sparql kernel](https://github.com/paulovn/sparql-kernel) instead of the Python3 kernel


We are going to use the second option to avoid installing new packages.

To use the library, you need to:

1. Import `sparql` from helpers (i.e., `helpers.py`, a file that is available in the github repository)

In [2]:
from helpers import sparql

2. Use the `%%sparql` magic command to indicate the SPARQL endpoint and then the SPARQL code.

Let's try it with some real queries.

# Queries against DBPedia

We are going to execute a SPARQL query against DBPedia. This section is based on [[8](#8)].

First, we just create a query to retrieve arbitrary triples (subject, predicate, object) without any restriction (besides limiting the result to 10 triples).

In [3]:
%%sparql https://dbpedia.org/sparql

SELECT ?s ?p ?o
WHERE {
    ?s ?p ?o
}
LIMIT 10

s,p,o
http://www.openlinksw.com/schemas/virtrdf#DefaultQuadMap,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMap
http://www.openlinksw.com/schemas/virtrdf#TmpQuadMap,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMap
http://www.openlinksw.com/schemas/virtrdf#DefaultServiceMap,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMap
http://www.openlinksw.com/virtrdf-data-formats#default-iid,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#default-iid-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#default-iid-blank,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#default-iid-blank-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#default,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat


Well, it worked, but the results are not particulary interesting. 
Let's search for a famous football player, Fernando Torres.

To do so, we will search for entities whose English "human-readable representation" (i.e., label) matches "Fernando Torres":

In [5]:
%%sparql https://dbpedia.org/sparql

SELECT *
WHERE
     {
        ?athlete rdfs:label "Rafael Nadal"@en 
     }

athlete
http://dbpedia.org/resource/Category:Rafael_Nadal
http://dbpedia.org/resource/Rafael_Nadal


Great, we found the IRI of the node: `http://dbpedia.org/resource/Fernando_Torres`

Now we can start asking for more properties.

To do so, go to http://dbpedia.org/resource/Fernando_Torres and you will see all the information available about Fernando Torres. Pay attention to the names of predicates to be able to create new queries. For example, we are interesting in knowing where Fernando Torres was born (`dbo:birthPlace`).

Let's go!

In [9]:
%%sparql https://dbpedia.org/sparql

PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT *
WHERE
     {
        ?athlete rdfs:label "Fernando Torres"@en ;
                 dbo:birthPlace ?CarreraDeportiva .       
     }

athlete,CarreraDeportiva
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada


If we examine the SPARQL query, we find three blocks:

* **PREFIX** section: IRIs of vocabularies and the prefix used below, to avoid long IRIs. e.g., by defining the `dbo` prefix in our example, the `dbo:birthPlace` below expands to `http://dbpedia.org/ontology/birthPlace`.
* **SELECT** section: variables we want to return (`*` is an abbreviation that selects all of the variables in a query)
* **WHERE** clause: triples where some elements are variables. These variables are bound during the query processing process and bounded variables are returned.

Now take a closer look at the **WHERE** section.
We said earlier that triples are made out of three elements and each triple pattern should finish with a  period (`.`) (although the last pattern can omit this).
However, when two or more triple patterns share the same subject, we omit it all but the first one, and use ` ;` as separator.
If if both the subject and predicate are the same, we could use a coma `,` instead.
This allows us to avoid repetition and make queries more readable.
But don't forget the space before your separators (`;` and `.`).

The result is interesting, we know he was born in Fuenlabrada, but we see an additional (wrong) value, the Spanish national football team. The conversion process from Wikipedia to DBPedia should still be tuned :).

We can *fix* it, by adding some more constaints.
In our case, only want a birth place that is also a municipality (i.e., its type is `http://dbpedia.org/resource/Municipalities_of_Spain`).
Let's see!

In [16]:
%%sparql https://dbpedia.org/sparql

PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>

SELECT *
WHERE
     {
        ?athlete rdfs:label "Fernando Torres"@en ;
                 dbo:birthPlace ?birthPlace .
        ?birthPlace dbo:type dbr:Municipalities_of_Spain 
     }

athlete,birthPlace
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada


Great. Now it looks better.
Notice that we added a new prefix.

Now, is Fuenlabrada is a big city?
Let's find out.

**Hint**: you can find more subject / object / predicate nodes related to [Fuenlabrada])http://dbpedia.org/resource/Fuenlabrada) in the RDF graph just as we did before.
That is how we found the `dbo:areaTotal` property.

In [17]:
%%sparql https://dbpedia.org/sparql

PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>

SELECT *
WHERE
     {
        dbr:Fuenlabrada dbo:areaTotal ?area 
     }

area
3.941e+07


Well, it shows 39.1 km$^2$.

Let's go back to our Fernando Torres.
What we are really insterested in is the name of the city he was born in, not its IRI.
As we saw before, the human-readable name is provided by the `rdfs:label` property:

In [22]:
%%sparql https://dbpedia.org/sparql

PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT *
WHERE
     {
        ?player rdfs:label "Fernando Torres"@en ;
                 dbo:birthPlace ?birthPlace .
        ?birthPlace dbo:type dbr:Municipalities_of_Spain ;
                    rdfs:label ?placeName        
                 
     }

player,birthPlace,placeName
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada,Fuenlabrada
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada,فوينلابرادا
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada,Fuenlabrada
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada,Fuenlabrada
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada,Fuenlabrada
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada,Φουενλαμπράδα
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada,Fuenlabrada
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada,Fuenlabrada
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada,Fuenlabrada
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada,Fuenlabrada


Well, we are almost there. We see that we receive the city name in many languages. We want just the English name. Let's filter them out!

In [19]:
%%sparql https://dbpedia.org/sparql

PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT *
WHERE
     {
        ?player rdfs:label "Fernando Torres"@en ;
                 dbo:birthPlace ?birthPlace .
        ?birthPlace dbo:type dbr:Municipalities_of_Spain ;
                    rdfs:label ?placeName .
         FILTER ( LANG ( ?placeName ) = 'en' )
                 
     }

player,birthPlace,placeName
http://dbpedia.org/resource/Fernando_Torres,http://dbpedia.org/resource/Fuenlabrada,Fuenlabrada


Awesome!

But we said we don't care about the IRI of the place. We only want two pieces of data: Fernando's birth date and the name of his birthplace.

Let's tune our query a bit more.

In [28]:
%%sparql https://dbpedia.org/sparql

PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT ?Nombre ,?birthDate, ?placeName
WHERE
     {
        ?player rdfs:label "Fernando Torres"@en ;
                 dbp:fullname ?Nombre ;
                 dbo:birthDate ?birthDate ;
                 dbo:birthPlace ?birthPlace .
        ?birthPlace dbo:type dbr:Municipalities_of_Spain ;
                    rdfs:label ?placeName .
         FILTER ( LANG ( ?placeName ) = 'en' )
                 
     }

Nombre,birthDate,placeName
Fernando José Torres Sanz,1984-03-20,Fuenlabrada


Great 😃

Are there many football players born in Fuenlabrada? Let's find out!

In [ ]:
%%sparql https://dbpedia.org/sparql

PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT *
WHERE
     {
        ?player a dbo:SoccerPlayer ;  
                  dbo:birthPlace dbr:Fuenlabrada .         
     }

Well, not that many. Observe we have used `a`.
It is just an abbreviation for `rdf:type`, both can be used interchangeably.

If you want additional examples, you can follow the notebook by [Shawn Graham](https://github.com/o-date/sparql-and-lod/blob/master/sparql-intro.ipynb), which is based on the  SPARQL tutorial by Matthew Lincoln, available [here in English](https://programminghistorian.org/en/lessons/retired/graph-databases-and-SPARQL) and [here in Spanish](https://programminghistorian.org/es/lecciones/retirada/sparql-datos-abiertos-enlazados]). You have also a local copy of these tutorials together with this notebook [here in English](https://htmlpreview.github.io/?https://github.com/gsi-upm/sitc/blob/master/lod/tutorial/graph-databases-and-SPARQL.html) and [here in Spanish](https://htmlpreview.github.io/?https://github.com/gsi-upm/sitc/blob/master/lod/tutorial/sparql-datos-abiertos-enlazados.html). 


## References

* <a id="1">[1]</a> [SPARQL by Example. A Tutorial. Lee Feigenbaum. W3C, 2009](https://www.w3.org/2009/Talks/0615-qbe/#q1)
* <a id="2">[2]</a> [RDF Primer W3C](https://www.w3.org/TR/rdf11-primer/)
* <a id="3">[3]</a> [SPARQL queries of Beatles recording sessions](http://www.snee.com/bobdc.blog/2017/11/sparql-queries-of-beatles-reco.html)
* <a id="4">[4]</a> [RDFLib documentation](https://rdflib.readthedocs.io/en/stable/).
* <a id="5">[5]</a> [Wikidata Query Service query examples](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)
* <a id="6">[6]</a> [RDF Graph Data Model. Learn about the RDF graph model used by Stardog.](https://www.stardog.com/tutorials/data-model)
* <a id="7">[7]</a> [Learn SPARQL Write Knowledge Graph queries using SPARQL with step-by-step examples.](https://www.stardog.com/tutorials/sparql/)
* <a id="8">[8]</a> [Running Basic SPARQL Queries Against DBpedia.](https://medium.com/virtuoso-blog/dbpedia-basic-queries-bc1ac172cc09)
* <a id="8">[9]</a> [Intro SPARQL based on painters.](https://github.com/o-date/sparql-and-lod/blob/master/sparql-intro.ipynb).

## Licence
The notebook is freely licensed under under the [Creative Commons Attribution Share-Alike license](https://creativecommons.org/licenses/by/2.0/).  

©  Carlos A. Iglesias, Universidad Politécnica de Madrid.